<a href="https://colab.research.google.com/github/MaLuisa7/BusinessIntelligence_Class_master/blob/main/timeseries_bi_20231023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
""" PRIMERO SE GENERO ESTO PARA CADA PRODUCTO Y COMPETIDOR

# Load the necessary libraries
library(forecast)

path <- "C:/Users/Usuario/Documents/BI/competidores/Competidor9.csv"
path_exp <- "C:/Users/Usuario/Documents/BI/competidores/Fcast_product2Competidor9.csv"

# Read the CSV file into a data frame
Competitor1 <- read.csv(path, header = TRUE, sep = ",")

# Create a time series from the data
ts.Competidor1 <- ts(Competitor1$Product2, frequency = 52, start = c(2019, 1))

# Plot the original time series
plot(ts.Competidor1)

# Fit an ARIMA model with automatic order selection
ARIMA.Competidor1 <- auto.arima(ts.Competidor1, D = 1)
summary(ARIMA.Competidor1)
plot(ARIMA.Competidor1$x, col = "blue")
par(new = TRUE)
plot(ARIMA.Competidor1$fitted, col = "red")
title(main = "Real vs Forecast")

# Create a forecast with 30 observations
Forecast.Competidor1 <- forecast(ARIMA.Competidor1, h = 30, level = 99)
print(Forecast.Competidor1)

# Extend the time index to include 30 additional periods
extended_time_index <- time(ts.Competidor1) + 30/52

# Create an extended time series with the forecasted values
extended_ts_Competidor1 <- ts(append(ts.Competidor1, rep(NA, 30)), frequency = 52, start = c(2019, 1))
extended_ts_Competidor1[(length(ts.Competidor1) + 1):(length(ts.Competidor1) + 30)] <- Forecast.Competidor1$mean

# Plot the extended time series
plot(extended_ts_Competidor1, main = "Competidor1 Forecasting", ylab = "Competidor1 Volume")

##--- export csv


# Export the time series to a CSV file in a different location
write.csv(data.frame(Time = time(extended_ts_Competidor1),
                     Value = as.vector(extended_ts_Competidor1)),
          file = path_exp, row.names = FALSE)


"""

' PRIMERO SE GENERO ESTO PARA CADA PRODUCTO Y COMPETIDOR\n \n# Load the necessary libraries\nlibrary(forecast)\n\npath <- "C:/Users/Usuario/Documents/BI/competidores/Competidor9.csv"\npath_exp <- "C:/Users/Usuario/Documents/BI/competidores/Fcast_product2Competidor9.csv"\n\n# Read the CSV file into a data frame\nCompetitor1 <- read.csv(path, header = TRUE, sep = ",")\n\n# Create a time series from the data\nts.Competidor1 <- ts(Competitor1$Product2, frequency = 52, start = c(2019, 1))\n\n# Plot the original time series\nplot(ts.Competidor1)\n\n# Fit an ARIMA model with automatic order selection\nARIMA.Competidor1 <- auto.arima(ts.Competidor1, D = 1)\nsummary(ARIMA.Competidor1)\nplot(ARIMA.Competidor1$x, col = "blue")\npar(new = TRUE)\nplot(ARIMA.Competidor1$fitted, col = "red")\ntitle(main = "Real vs Forecast")\n\n# Create a forecast with 30 observations\nForecast.Competidor1 <- forecast(ARIMA.Competidor1, h = 30, level = 99)\nprint(Forecast.Competidor1)\n\n# Extend the time index to in

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pmdarima as pm
from pmdarima import model_selection
import numpy as np
from matplotlib import pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

In [3]:

def datapreprocesssing_oneprod(df_c, path_p1,  competidor):

  fcast_c1_p1 = pd.read_csv(path_p1)
  # fcast_c1_p2 = pd.read_csv(path_p2)
  fcast_c1_p1.rename(columns={'Value': 'producto1'}, inplace= True)
  # fcast_c1_p2.rename(columns={'Value': 'producto2'}, inplace= True)

  #data prerpocessing
  df = fcast_c1_p1 # pd.concat([fcast_c1_p1,fcast_c1_p2], axis=1)
  df.drop(columns = ['Time'], inplace=True)
  df["Time"] = list(range(1, len(df)+1))
  df1 = df.merge(df_c, on ='Time', how='left')
  df1.drop(columns = ['Product1' ], inplace=True)
  df1 = df1.query("(YMD != 20200631) and (YMD !=20210231)")
  df1['YMD']= df1.loc[:,'YMD'].astype('str')
  df1['YMD'] = pd.to_datetime(df1['YMD'], format='%Y%m%d', errors='coerce')
  last_valid_date = df1['YMD'].dropna().iloc[-1]# Find the last valid date in the 'YMD' column
  date_range = pd.date_range(start=last_valid_date + pd.DateOffset(7), periods=30, freq='W')# Generate a date range for the missing dates by week
  df1['YMD'].loc[df1['YMD'].isna()] = date_range# Fill in missing dates by week
  df1['tipo']=np.where(df1.loc[:,'YMD'] <=last_valid_date, 'observed','forecasted')
  df1['year']= df1.YMD.dt.year
  df1['month']= df1.YMD.dt.month
  df1['day']= df1.YMD.dt.day
  df1['competidor'] = competidor

  return df1

In [4]:

def datapreprocesssing_twoprods(df_c, path_p1, path_p2, competidor):

  fcast_c1_p1 = pd.read_csv(path_p1)
  fcast_c1_p2 = pd.read_csv(path_p2)
  fcast_c1_p1.rename(columns={'Value': 'producto1'}, inplace= True)
  fcast_c1_p2.rename(columns={'Value': 'producto2'}, inplace= True)

  #data prerpocessing
  df = pd.concat([fcast_c1_p1,fcast_c1_p2], axis=1)
  df.drop(columns = ['Time'], inplace=True)
  df["Time"] = list(range(1, len(df)+1))
  df1 = df.merge(df_c, on ='Time', how='left')
  df1.drop(columns = ['Product1', "Product2"], inplace=True)
  df1 = df1.query("(YMD != 20200631) and (YMD !=20210231)")
  df1['YMD']= df1.loc[:,'YMD'].astype('str')
  df1['YMD'] = pd.to_datetime(df1['YMD'], format='%Y%m%d', errors='coerce')
  last_valid_date = df1['YMD'].dropna().iloc[-1]# Find the last valid date in the 'YMD' column
  date_range = pd.date_range(start=last_valid_date + pd.DateOffset(7), periods=30, freq='W')# Generate a date range for the missing dates by week
  df1['YMD'].loc[df1['YMD'].isna()] = date_range# Fill in missing dates by week
  df1['tipo']=np.where(df1.loc[:,'YMD'] <=last_valid_date, 'observed','forecasted')
  df1['year']= df1.YMD.dt.year
  df1['month']= df1.YMD.dt.month
  df1['day']= df1.YMD.dt.day
  df1['competidor'] = competidor

  return df1

In [5]:
c1 = pd.read_csv("/content/Competidor1.csv", sep= ",", header=0)
c2 = pd.read_csv("/content/Competidor2.csv", sep= ",", header=0)
c3 = pd.read_csv("/content/Competidor3.csv", sep= ",", header=0)
c4 = pd.read_csv("/content/Competidor4.csv", sep= ",", header=0)
c5 = pd.read_csv("/content/Competidor5.csv", sep= ",", header=0)
c6 = pd.read_csv("/content/Competidor6.csv", sep= ",", header=0)
c7 = pd.read_csv("/content/Competidor7.csv", sep= ",", header=0)
c8 = pd.read_csv("/content/Competidor8.csv", sep= ",", header=0)
c9 = pd.read_csv("/content/Competidor9.csv", sep= ",", header=0)
c10= pd.read_csv("/content/Competidor10.csv")

In [6]:
dicst_vars = {"c1":c1,"c2":c2,"c3":c3,"c4":c4,"c5":c5,"c6":c6,"c7":c7,"c8":c8,"c9":c9,"c10":c10}

In [21]:
lst_dfs =[]
for i in range(1, 11):
  if i%2 !=0:
    path1 = f"/content/Fcast_product1Competidor{i}.csv"
    path2= f"/content/Fcast_product2Competidor{i}.csv"
    comp = dicst_vars.get(f'c{i}')
    print(f'c{i}')
    df_res =datapreprocesssing_twoprods(comp, path1, path2, f'competidor {i}')
    lst_dfs.append(df_res)
  else:
    path1 = f"/content/Fcast_product1Competidor{i}.csv"
    comp = dicst_vars.get(f'c{i}')
    print(f'c{i}')
    df_res = datapreprocesssing_oneprod(comp, path1,  f'competidor {i}')

    lst_dfs.append(df_res)

c1
c2
c3
c4
c5
c6
c7
c8
c9
c10


In [24]:
dict_comp = {'competidor 1': 'Bohemia',
             'competidor 3': "Indio",
             'competidor 2': "Corona",
             'competidor 9': "Modelo",
             'competidor 5': "Tecate",
             'competidor 6': "Stella Artois" ,
             'competidor 7': "Victoria" ,
             'competidor 8': "Michelob Ultra",
             'competidor 4': "Barrilito",
             'competidor 10': "Sol"}

In [27]:
dfres = pd.concat(lst_dfs)
dfres['volumen_total'] = np.where( ~dfres.producto2.isna(), dfres.producto1 + dfres.producto2, dfres.producto1)
dfres.replace(dict_comp, inplace=True)

In [32]:
df2 = dfres.groupby(['competidor','tipo']).sum().loc[:,['producto1', 'producto2','volumen_total']].reset_index()
df2.sort_values(by =['tipo','volumen_total'], inplace=True, ascending=False)
display(df2)
df_topventas = df2.copy()

,competidor,tipo,producto1,producto2,volumen_total
15,Stella Artois,observed,9225742386.04,0.00,9225742386.04
5,Corona,observed,7380060264.12,0.00,7380060264.12
1,Barrilito,observed,6160840544.89,0.00,6160840544.89
11,Modelo,observed,544614123.00,2849388006.50,3394002129.50
17,Tecate,observed,656829465.70,2353481076.70,3010310542.40
3,Bohemia,observed,374727221.70,2283350598.70,2658077820.40
9,Michelob Ultra,observed,2608526195.14,0.00,2608526195.14
7,Indio,observed,508855998.00,1173272181.70,1682128179.70
19,Victoria,observed,209157364.70,447026569.70,656183934.40
13,Sol,observed,544614123.00,0.00,544614123.00


In [37]:
df3 = dfres.groupby(['tipo','month','competidor']).sum().loc[:,['producto1', 'producto2','volumen_total']].reset_index()
df3.sort_values(by =['tipo','month','competidor'], inplace=True, ascending=False)


In [38]:
df_top_mes = df3.copy()

,tipo,month,competidor,producto1,producto2,volumen_total
199,observed,12,Victoria,17961100.00,35839220.90,53800320.90
198,observed,12,Tecate,57398963.00,192247155.40,249646118.40
197,observed,12,Stella Artois,767870930.80,0.00,767870930.80
196,observed,12,Sol,46339476.00,0.00,46339476.00
195,observed,12,Modelo,46339476.00,230326149.20,276665625.20
...,...,...,...,...,...,...
4,forecasted,5,Michelob Ultra,43336903.26,0.00,43336903.26
3,forecasted,5,Indio,8638340.81,16850062.26,25488403.06
2,forecasted,5,Corona,116878321.59,0.00,116878321.59
1,forecasted,5,Bohemia,5922877.66,37233161.65,43156039.31


In [ ]:
df_topventas
df_top_mes
dfres